# Feature 별 가중치 및 흡연위험지수 산출

흡연위험지수의 산출을 위해 최종 변수로 설정된 feature들 각각의 가중치를 산정해야함.  
변수는 크게 `흡연발생변수`와 `취약계층변수` 두 가지로 구분함.  
`흡연발생변수`는 상관관계 분석을 통해 산출하며 `취약계층변수`는 연령대를 기준으로 차등적 점수를 책정함.

## 라이브러리 및 환경 세팅

In [1]:
import numpy as np
import pandas as pd
pd.set_option("display.max_rows", 100, "display.max_columns", 50)
pd.set_option('mode.chained_assignment',  None)

In [2]:
PATH = 'G:/다른 컴퓨터/My_desktop/project/데이터 분석 프로젝트/이노포스트/data/전처리 완료 데이터/'
# PATH = 'D:/project/데이터 분석 프로젝트/이노포스트/data/전처리 완료 데이터/'
# PATH = '/content/drive/MyDrive/이노포스트/data/전처리 완료 데이터/'

In [3]:
# colab용 구글 드라이브 마운트
# from google.colab import drive
# drive.mount('/content/drive')

## 데이터 로드

In [4]:
df = pd.read_pickle(PATH + '흡연발생변수.pkl')

In [5]:
# import pickle5 as pickle

# with open(PATH+'흡연발생변수.pkl', 'rb') as fh:
#    df = pickle.load(fh)

In [6]:
df.head()

,행정동,흡연지수,과태료,음식점,술집&유흥주점,PC방&오락&레저,목욕장,공원&놀이터,노상주차장,흡연인구
0,가수원동,0,17,126,11,0,1,2,0,2803
1,가양1동,0,10,163,26,4,1,2,6,4271
2,가양2동,0,25,266,24,7,2,2,6,5554
3,가장동,0,59,92,13,2,3,1,0,2819
4,갈마1동,0,91,306,48,13,0,0,2,5990


## 흡연발생지수 가중치 산정

수집된 데이터 중 흡연이 실제로 일어났다는 정보를 줄 수 있는 유일한 컬럼인 `과태료`를 $y$ 로 설정함  
`흡연발생변수` 들의 correlation을 진행하여 `과태료`를 기준으로 한 상관관계를 가중치로 산정함

In [7]:
corr = df.corr(method='pearson')
corr = corr.iloc[1:,1:]
corr

,과태료,음식점,술집&유흥주점,PC방&오락&레저,목욕장,공원&놀이터,노상주차장,흡연인구
과태료,1.000000,0.278961,0.359845,0.280555,0.071887,-0.153977,0.032758,0.258411
음식점,0.278961,1.000000,0.878924,0.859056,0.513741,0.337083,0.257269,0.688433
술집&유흥주점,0.359845,0.878924,1.000000,0.783716,0.597860,0.115308,0.233917,0.517957
PC방&오락&레저,0.280555,0.859056,0.783716,1.000000,0.517821,0.296528,0.249877,0.635887
목욕장,0.071887,0.513741,0.597860,0.517821,1.000000,0.041779,0.345636,0.215772
공원&놀이터,-0.153977,0.337083,0.115308,0.296528,0.041779,1.000000,0.232459,0.595651
노상주차장,0.032758,0.257269,0.233917,0.249877,0.345636,0.232459,1.000000,0.275824
흡연인구,0.258411,0.688433,0.517957,0.635887,0.215772,0.595651,0.275824,1.000000


음의 상관관계를 보이는 피쳐가 발견됨  
    1. `공원&놀이터` 를 취약계층 관련 변수로 변경    

처음 용어 정의 시 공원이나 놀이터를 `공공시설`로 분류하고 이용 인구가 많은 `공공시설은 흡연발생 위험이 높다`라고 가정했으나 음의 상관관계를 보여 흡연발생변수에서 기각.  
그러나 공원과 놀이터는 흡연취약계층이 이용하는 시설로도 분류가 가능하여 3번으로 진행 결정함

In [8]:
corr.drop(['공원&놀이터'], inplace=True)

In [9]:
df.drop(['공원&놀이터'], axis=1, inplace=True)

In [10]:
weight = corr['과태료']*100
weight

과태료          100.000000
음식점           27.896061
술집&유흥주점       35.984496
PC방&오락&레저     28.055463
목욕장            7.188721
노상주차장          3.275826
흡연인구          25.841083
Name: 과태료, dtype: float64

## 스케일링

각 피쳐 별 표본수 차이로 인해 가중치가 왜곡되어 반영되는 것을 방지하기 위해 스케일링이 필요함.  
데이터에 이상치가 없어 `feature별 총 합`을 분모로 각 `행정동 별 수치`를 분자로 하여 비중으로 스케일링 진행함.

In [11]:
df.sum()

행정동          가수원동가양1동가양2동가장동갈마1동갈마2동관저1동관저2동관평동괴정동구즉동기성동내동노...
흡연지수                                                         0
과태료                                                       2804
음식점                                                      20835
술집&유흥주점                                                   2797
PC방&오락&레저                                                  556
목욕장                                                        116
노상주차장                                                      496
흡연인구                                                    418140
dtype: object

In [12]:
df1 = df.iloc[:,2:]
df1.head(3)

,과태료,음식점,술집&유흥주점,PC방&오락&레저,목욕장,노상주차장,흡연인구
0,17,126,11,0,1,0,2803
1,10,163,26,4,1,6,4271
2,25,266,24,7,2,6,5554


In [13]:
# 피쳐별 x번째 행 해당값의 비율
def ratio(x):
      return (df1.iloc[x,:]/df1.sum())

In [14]:
# summation of weighted x for each row # x번째 행의 가중치반영된 피쳐값들의 합
def sum_weighted(x):
      return sum(weight*ratio(x)) 

In [15]:
# 계산이 잘 되는지 샘플 확인
sum(weight*ratio(0))

1.1516952619756005

In [16]:
df.head()

,행정동,흡연지수,과태료,음식점,술집&유흥주점,PC방&오락&레저,목욕장,노상주차장,흡연인구
0,가수원동,0,17,126,11,0,1,0,2803
1,가양1동,0,10,163,26,4,1,6,4271
2,가양2동,0,25,266,24,7,2,6,5554
3,가장동,0,59,92,13,2,3,0,2819
4,갈마1동,0,91,306,48,13,0,2,5990


In [17]:
df['흡연발생지수'] = np.nan
sum_list = []

for i in range(len(df1)):
  sum_list.append(sum_weighted(i)) # 리스트 i번째 원소값을 바로 넣으려했더니 안 들어가져서 빈 리스트 만들어서 '흡연지수_1' 컬럼에 넣음

df['흡연발생지수'] = sum_list

In [18]:
df.head()

,행정동,흡연지수,과태료,음식점,술집&유흥주점,PC방&오락&레저,목욕장,노상주차장,흡연인구,흡연발생지수
0,가수원동,0,17,126,11,0,1,0,2803,1.151695
1,가양1동,0,10,163,26,4,1,6,4271,1.476759
2,가양2동,0,25,266,24,7,2,6,5554,2.416526
3,가장동,0,59,92,13,2,3,0,2819,2.855615
4,갈마1동,0,91,306,48,13,0,2,5990,5.311972


## 취약계층 변수 가중치

In [19]:
weak = pd.read_pickle(PATH + '취약계층변수.pkl')

In [20]:
#with open(PATH+'취약계층변수.pkl', 'rb') as fh:
#   weak = pickle.load(fh)

In [21]:
weak.head()

,행정동,공원&놀이터,영유아시설,산부인과&소아청소년과,임산부,유치원생시설,초등학생시설,중고등학생시설,미성년인구
0,가수원동,2,11,0,43,12,11,3,1523
1,가양1동,2,7,0,36,7,7,4,1646
2,가양2동,2,9,1,54,14,14,20,2444
3,가장동,1,12,0,41,8,7,14,1682
4,갈마1동,0,15,0,74,23,22,18,2510


In [22]:
weak = weak[['공원&놀이터',
             '영유아시설',
             '산부인과&소아청소년과',
             '임산부',
             '유치원생시설',
             '초등학생시설',
             '중고등학생시설',
             '미성년인구']]

In [23]:
weak.head()

,공원&놀이터,영유아시설,산부인과&소아청소년과,임산부,유치원생시설,초등학생시설,중고등학생시설,미성년인구
0,2,11,0,43,12,11,3,1523
1,2,7,0,36,7,7,4,1646
2,2,9,1,54,14,14,20,2444
3,1,12,0,41,8,7,14,1682
4,0,15,0,74,23,22,18,2510


In [24]:
weak_weight = [0.389, 0.311, 0.311, 0.311, 0.233, 0.156, 0.078, 0.778]

In [25]:
def ratio(x):
      return (weak.iloc[x,:]/weak.sum()) # 피쳐별 x번째 행 해당값의 비율


In [26]:
def sum_weighted(x):
      return sum(weak_weight*ratio(x))  # summation of weighted x for each row # x번째 행의 가중치반영된 피쳐값들의 합

In [27]:
sum(weak_weight*ratio(0))

0.013852932861034966

In [28]:
df['취약계층지수'] = np.nan
sum_list = []

for i in range(len(weak)):
  sum_list.append(sum_weighted(i)) # 리스트 i번째 원소값을 바로 넣으려했더니 안 들어가져서 빈 리스트 만들어서 '취약지수' 컬럼에 넣음

df['취약계층지수'] = sum_list

In [29]:
df.head()

,행정동,흡연지수,과태료,음식점,술집&유흥주점,PC방&오락&레저,목욕장,노상주차장,흡연인구,흡연발생지수,취약계층지수
0,가수원동,0,17,126,11,0,1,0,2803,1.151695,0.013853
1,가양1동,0,10,163,26,4,1,6,4271,1.476759,0.011705
2,가양2동,0,25,266,24,7,2,6,5554,2.416526,0.021863
3,가장동,0,59,92,13,2,3,0,2819,2.855615,0.013619
4,갈마1동,0,91,306,48,13,0,2,5990,5.311972,0.022509


## 흡연위험지수 산출

흡연위험지수(score) = (Scaled_흡연발생변수 * 변수별 가중치)+ (Scaled_취약계층변수 * 변수별 가중치)  

In [30]:
final = pd.DataFrame(columns=['행정동', '흡연발생지수', '취약계층지수'])
final['행정동'] = df['행정동']
final['흡연발생지수'] = df['흡연발생지수']
final['취약계층지수'] = df['취약계층지수']
final.head()

,행정동,흡연발생지수,취약계층지수
0,가수원동,1.151695,0.013853
1,가양1동,1.476759,0.011705
2,가양2동,2.416526,0.021863
3,가장동,2.855615,0.013619
4,갈마1동,5.311972,0.022509


In [34]:
gu_dong = pd.read_csv('G:/다른 컴퓨터/My_desktop/project/데이터 분석 프로젝트/이노포스트/data/행정구역별_위경도_좌표_대전만.csv', encoding='cp949')
gu_dong.head()

,시도,시군구,행정동,위도,경도,흡연발생지수
0,대전광역시,서구,가수원동,36.303513,127.349430,1.151695
1,대전광역시,동구,가양1동,36.347013,127.441410,1.476759
2,대전광역시,동구,가양2동,36.349057,127.447397,2.416526
3,대전광역시,서구,가장동,36.330132,127.385195,2.855615
4,대전광역시,서구,갈마1동,36.350845,127.367834,5.311972


In [35]:
len(gu_dong)

82

In [36]:
gu_dong = gu_dong.iloc[:,1:3]

In [37]:
final['구'] = np.nan

for i in range(len(final)):
  for j in range(len(gu_dong)):
    if final['행정동'][i] == gu_dong['행정동'][j]:
      final['구'][i] = gu_dong['시군구'][j]

In [38]:
final = final[['구','행정동','흡연발생지수','취약계층지수']]
final['흡연위험지수'] = final['흡연발생지수'] + final['취약계층지수']
final.head()

,구,행정동,흡연발생지수,취약계층지수,흡연위험지수
0,서구,가수원동,1.151695,0.013853,1.165548
1,동구,가양1동,1.476759,0.011705,1.488465
2,동구,가양2동,2.416526,0.021863,2.438389
3,서구,가장동,2.855615,0.013619,2.869233
4,서구,갈마1동,5.311972,0.022509,5.334481


In [39]:
final.to_pickle(PATH + '최종지수도출.pkl')
final.to_csv(PATH + '최종지수도출.csv', index=False, encoding='cp949')